# **Dungeon and Fighter Crawling**
* 던전앤파이터 공식 홈페이지에서 직업, 명성을 바탕으로 캐릭터 정보를 크롤링합니다.
* https://df.nexon.com/world/fame

### **MySQL DB에 연결**

In [1]:
from src.dnf_driver import DNF_DRIVER
from configs.config import MYSQL_CONNECTION_STRING
from configs.config import DATA_PATH
from datetime import datetime, timedelta
from glob import glob
import pandas as pd
import os

%load_ext sql
%sql {MYSQL_CONNECTION_STRING}

### **직업 정보 가져오기**
* job_info에서 크롤링을 원하는 직업 가져오기.


In [ ]:
query = """
select *
from job_info
"""
request = %sql {query}
request_list = list(request)
request_list

### 크롤링 실행하기

In [ ]:

# * 오전 06:00 전 데이터는 이전 날짜로 취급합니다.
date = (datetime.now() - timedelta(hours=6)).strftime('%Y%m%d')
folder_path = os.path.join(DATA_PATH, 'crawling_data', date)

dd = DNF_DRIVER()
for job_name, job_group, job_code in request_list:
    
    df = dd.crawling(job_group, job_name, job_code, 66000,66500)
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, f'{job_name}.csv')
    df.to_csv(file_path, encoding='utf-8', index=False)


### 결과 데이터 통합 후 저장하기

In [ ]:
csv_files = glob(os.path.join(folder_path, '*.csv'))

if len(csv_files) == 67:
    result = pd.DataFrame()
    for file in csv_files:
        df = pd.read_csv(file)
        result = pd.concat([result, df], ignore_index=True)

    file_path = os.path.join(folder_path, 'character.csv')
    result.to_csv(file_path, encoding='utf-8', index=False)